In [1]:
#pandas pour travailler sur les tableau de donnes
import pandas as pd
#connenction a mongo
import pymongo

#connection a mongo
con = pymongo.MongoClient()
#connection a la database pharmacie
db = con['pharmacie']


# insertions Pharmacies

In [69]:
def read_pharmacie(nom):
    index=['nom','telephone','addresse_postal','addresse', 'assurance', 'x','y']
    data= pd.read_excel("Goupes_pharmacies_0.xlsx",sheet_name=nom)
    data.columns=index
    for x in data.columns:
        if x not in ['addresse','x', 'y','telephone']:
            data[x]=data[x].astype(str)
            data[x]=data[x].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
            data[x]= data[x].str.strip()
            data[x]= data[x].str.lower()
        if x in ['x','y']:
            data[x]=data[x].astype(float)
        if x=='telephone':
            data[x]=data[x].astype(int)
    data['groupe']=nom
    return data

In [70]:
d=read_pharmacie('groupe_2')
d.dtypes

nom                 object
telephone            int64
addresse_postal     object
addresse            object
assurance           object
x                  float64
y                  float64
groupe              object
dtype: object

In [71]:
data=pd.concat([read_pharmacie('groupe_1'), read_pharmacie('groupe_2'), read_pharmacie('groupe_3'), read_pharmacie('groupe_4')], ignore_index=True)
data=data.to_dict(orient='records')
data

[{'nom': 'avenir',
  'telephone': 25361338,
  'addresse_postal': '01 bp 4881',
  'addresse': 'Secteur 14 (Zogona) - Avenue Babanguida, Ouagadougou',
  'assurance': 'sonar, sunu, uab',
  'x': -1.49388,
  'y': 12.37513,
  'groupe': 'groupe_1'},
 {'nom': 'baowendsom',
  'telephone': 25414499,
  'addresse_postal': 'nan',
  'addresse': "Tampouy, secteur 15, sur le goudron du Collège Notre Dame de l'Espérance",
  'assurance': 'nan',
  'x': -1.577449,
  'y': 12.403442,
  'groupe': 'groupe_1'},
 {'nom': 'barkwende',
  'telephone': 25408590,
  'addresse_postal': 'nan',
  'addresse': 'Arrondissement 8, secteur 35, face à la cité de Rimkieta',
  'assurance': 'nan',
  'x': -1.608512,
  'y': 12.37372,
  'groupe': 'groupe_1'},
 {'nom': 'beatitudes',
  'telephone': 25374711,
  'addresse_postal': 'nan',
  'addresse': 'Secteur 15 (Ouaga 2000) - Boulevard France-Afrique, Ouagadougou',
  'assurance': 'coris, mci, sonar, sunu',
  'x': -1.528543,
  'y': 12.307399,
  'groupe': 'groupe_1'},
 {'nom': 'benaia'

In [72]:
for pharm in data:
    pharm['localisation']={
        "type":"Point",
        "coordinates":[pharm['x'],pharm['y']]
    }
    del pharm['x']
    del pharm['y']
data

[{'nom': 'avenir',
  'telephone': 25361338,
  'addresse_postal': '01 bp 4881',
  'addresse': 'Secteur 14 (Zogona) - Avenue Babanguida, Ouagadougou',
  'assurance': 'sonar, sunu, uab',
  'groupe': 'groupe_1',
  'localisation': {'type': 'Point', 'coordinates': [-1.49388, 12.37513]}},
 {'nom': 'baowendsom',
  'telephone': 25414499,
  'addresse_postal': 'nan',
  'addresse': "Tampouy, secteur 15, sur le goudron du Collège Notre Dame de l'Espérance",
  'assurance': 'nan',
  'groupe': 'groupe_1',
  'localisation': {'type': 'Point', 'coordinates': [-1.577449, 12.403442]}},
 {'nom': 'barkwende',
  'telephone': 25408590,
  'addresse_postal': 'nan',
  'addresse': 'Arrondissement 8, secteur 35, face à la cité de Rimkieta',
  'assurance': 'nan',
  'groupe': 'groupe_1',
  'localisation': {'type': 'Point', 'coordinates': [-1.608512, 12.37372]}},
 {'nom': 'beatitudes',
  'telephone': 25374711,
  'addresse_postal': 'nan',
  'addresse': 'Secteur 15 (Ouaga 2000) - Boulevard France-Afrique, Ouagadougou',


In [74]:
db['pharmacies'].insert_many(data);

# insertion produits

In [37]:
def read_produit(nom,pharmacie):
    index=['nom','dci','dosage','forme_galenique', 'labo_fabricant', 'conditionnement','pu']
    data= pd.read_excel(nom,sheet_name="Base de données détaillées")
    data.columns=index
    data['pharmacie']=pharmacie
    for x in data.columns:
        if x !='pu':
            data[x]=data[x].astype(str)
            data[x]=data[x].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
            data[x]= data[x].str.strip()
            data[x]= data[x].str.lower()

    
    return data

In [38]:
liste= pd.concat([read_produit("Listing produits Phcie  Georgette.xlsx", "georgette"),read_produit("Listing produits Phcie  Hamamickely.xls", "hamamickely")], ignore_index=True)
liste

,nom,dci,dosage,forme_galenique,labo_fabricant,conditionnement,pu,pharmacie
0,algesal suractive,"aspirine, diethylamine, myrtecaine","6,54g/3,46g/1g",creme,nan,t/40g,2650.0,georgette
1,algesic extra,"paracetamol,codeine",500mg/65mg,cpr pell sec,nan,boite/16,1000.0,georgette
2,allergodil,azelastine,0.0005,collyre,nan,f/6ml,5025.0,georgette
3,alvityl,multivitamines,nan,sirop,nan,fl/150ml,4950.0,georgette
4,alvityl tonus,multivitamines,nan,cp eff,nan,boite/10,2600.0,georgette
5,amaday,"amlodipine,perindopril",10/4mg,cp,nan,boite/30,8700.0,georgette
6,amaday,amlodipine,10mg,cp,nan,boite/30,7000.0,georgette
7,captopril tm,captopril,25mg,cp,tongmei,boite/100,4250.0,georgette
8,captopril tm,captopril,25mg,cp,tongmei,blister/20,850.0,georgette
9,captopril denk,captopril,25mg,cp,denk pharma,b/100,7725.0,georgette


In [39]:
liste=liste.to_dict(orient='records')
liste

[{'nom': 'algesal suractive',
  'dci': 'aspirine, diethylamine, myrtecaine',
  'dosage': '6,54g/3,46g/1g',
  'forme_galenique': 'creme',
  'labo_fabricant': 'nan',
  'conditionnement': 't/40g',
  'pu': 2650.0,
  'pharmacie': 'georgette'},
 {'nom': 'algesic extra',
  'dci': 'paracetamol,codeine',
  'dosage': '500mg/65mg',
  'forme_galenique': 'cpr pell sec',
  'labo_fabricant': 'nan',
  'conditionnement': 'boite/16',
  'pu': 1000.0,
  'pharmacie': 'georgette'},
 {'nom': 'allergodil',
  'dci': 'azelastine',
  'dosage': '0.0005',
  'forme_galenique': 'collyre',
  'labo_fabricant': 'nan',
  'conditionnement': 'f/6ml',
  'pu': 5025.0,
  'pharmacie': 'georgette'},
 {'nom': 'alvityl',
  'dci': 'multivitamines',
  'dosage': 'nan',
  'forme_galenique': 'sirop',
  'labo_fabricant': 'nan',
  'conditionnement': 'fl/150ml',
  'pu': 4950.0,
  'pharmacie': 'georgette'},
 {'nom': 'alvityl tonus',
  'dci': 'multivitamines',
  'dosage': 'nan',
  'forme_galenique': 'cp eff',
  'labo_fabricant': 'nan',
  

In [40]:
def insertion_medicament(data):
    query=db['produits'].find_one({
        'nom':data['nom'],
        'dci':data['dci'], 
        'dosage': data['dosage'],
        'forme_galenique':data['forme_galenique'],
        'conditionnement':data['conditionnement']
    })
    id_pharm=db['pharmacies'].find_one({'nom':data['pharmacie']})
    
    if query:
        initial= query['pharmacies']
        initial.append({"_id":id_pharm['_id'],
                        "disponible":True,
                        'pu':data['pu']
        })
        myquery = { "nom": data['nom'] }
        newvalues = { "$set": { "pharmacies": initial } }
        db['produits'].update_one(myquery, newvalues)
        
        
    else:
        
        data['pharmacies']=[{
            "_id":id_pharm['_id'],
            "disponible":True,
            'pu':data['pu']
            }]
        del data['pharmacie']
        db['produits'].insert_one(data)

In [42]:
for line in liste:
    insertion_medicament(line)